In [1]:
import pandas as pd
import pysam as ps
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# all variants in +/- 25 bp region around exon-intron boundary (as dataframe available)
variants = pd.read_pickle('DataFrame_all_variants')
variants = variants.sort_values(by=['#CHROM', 'POS'], ignore_index=True)

### VEP annotations

In [14]:
# data from vep
vep = pd.read_table('vep_download.txt')

In [15]:
# some variants (often insertions) not with HGVSg in VEP --> in VCF format for VEP
not_in_vep = variants[~variants['HGVSg'].isin(list(vep['#Uploaded_variation']))].copy()
not_in_vep.drop_duplicates(subset=['#CHROM', 'POS', 'REF', 'ALT'], inplace=True, ignore_index=True)
not_in_vep['ID'] = not_in_vep['HGVSg']
not_in_vep[['#CHROM', 'POS', 'ID', 'REF', 'ALT']].to_csv('02_vep/vep_ins.txt', sep='\t', header=False, index=False)

In [16]:
# insertion variants
vep_ins = pd.read_table('02_vep/vep_ins.txt')
vep = vep.append(vep_ins, ignore_index=True)
vep[['transcr']] = vep['Feature'].str.split('.', expand=True)[0]
vep_ref = vep[vep['transcr'].isin(variants['RefSeq accession'])].copy()
vep_ref.drop_duplicates(inplace=True)
vep_ref = vep_ref.reset_index(drop=True)
vep_ref = vep_ref.replace('-', np.nan)

In [18]:
# one RefSeq not in vep_ref, but in variants (NM_020732) for gene ARID1B
# 25 unique variants are missing
# --> RefSeq has been replaced by NM_001374820
wrong_refseq = set(vep_ref['transcr']) ^ set(variants['RefSeq accession'])
wrong_refseq_vars = variants[variants['RefSeq accession']==list(wrong_refseq)[0]]
vep_wrong_refseq = vep[(vep['#Uploaded_variation'].isin(list(wrong_refseq_vars['HGVSg'])))&
                       (vep['SYMBOL']==wrong_refseq_vars['HUGO_Symbol'].unique()[0])]
vep_wrong_refseq = vep_wrong_refseq[vep_wrong_refseq['transcr']=='NM_001374820']
# change RefSeq to herein used RefSeq
new_refs = vep_wrong_refseq.copy()
new_refs['transcr'] = 'NM_020732'

In [19]:
vep_ref = vep_ref.append(new_refs, ignore_index=True)
vep_ref = vep_ref.rename(columns={'#Uploaded_variation':'HGVSg', 'transcr':'RefSeq accession'})

In [20]:
# merge VEP annotations to all variants
vars_vep = pd.merge(variants, vep_ref[['IMPACT', 'EXON', 'INTRON', 'cDNA_position', 'CDS_position',
                                        'Protein_position', 'Amino_acids', 'Codons', 'DISTANCE', 'STRAND', 
                                        'SIFT', 'PolyPhen', 'HGVS_OFFSET', 'SpliceAI_pred_DP_AG', 
                                        'SpliceAI_pred_DP_AL', 'SpliceAI_pred_DP_DG', 'SpliceAI_pred_DP_DL', 
                                        'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 
                                        'SpliceAI_pred_DS_DL', 'SpliceAI_pred_SYMBOL', 'CADD_PHRED', 'CADD_RAW', 
                                        'MaxEntScan_alt', 'MaxEntScan_diff', 'MaxEntScan_ref', 'ada_score', 
                                        'rf_score', 'HGVSg', 'RefSeq accession']], 
                    how='left', on=['HGVSg', 'RefSeq accession'])
vars_vep = vars_vep.drop_duplicates()

### FLOSSIES annotations

In [21]:
# previously as dataframe
floss = pd.read_pickle('DataFrame_flossies')
floss.rename(columns={'CHROM':'#CHROM'}, inplace=True)
for df in [floss, vars_vep]:
    df['POS'] = df['POS'].astype(int)
    df[['#CHROM','REF','ALT']] = df[['#CHROM','REF','ALT']].astype(str)

In [ ]:
vars_vep_floss = pd.merge(vars_vep, floss[['#CHROM', 'POS', 'REF', 'ALT', 'Splice_Change_Flossies', 
                                           'European_(n=7325)_Flossies', 'African_(n=2559)_Flossies',
                                           'Overall_Frequency_Flossies', 'European (n=3646)', 
                                           'African (n=1283)']], 
                          how='left', on=['#CHROM','POS','REF','ALT'])
vars_vep_floss = vars_vep_floss.replace('-', np.nan)
vars_vep_floss = vars_vep_floss.replace('.', np.nan)

### Variants of the HBOC consortium

In [14]:
# excel sheet of HBOC variant database
hboc = pd.read_excel('hboc_variants.xlsx')
hboc['POS'] = hboc['POS'].astype(int)
hboc[['#CHROM','REF','ALT']] = hboc[['#CHROM','REF','ALT']].astype(str)

In [15]:
# find and eliminate duplicates in this database
hboc2 = hboc.copy()
hboc2 = hboc2.reset_index(drop=False)
dupl2 = hboc2.groupby(['#CHROM', 'POS', 'REF', 'ALT'])
dupl_cpra2 = [i for i,df in dupl2 if len(df)>1]
#dupl2.get_group(dupl_cpra2[0])

In [16]:
for_drop = []
for i in dupl_cpra2:
    df = dupl2.get_group(i)
    if not df[df['ART']=='keine Angabe'].empty and len(df[df['ART']=='keine Angabe'])<2:
        for_drop.append(df[df['ART']=='keine Angabe'].iloc[0]['index'])
hboc2.drop(index=for_drop, inplace=True)

In [17]:
dupl3 = hboc2.groupby(['#CHROM', 'POS', 'REF', 'ALT'])
dupl_cpra3 = [i for i,df in dupl3 if len(df)>1]
#len(dupl_cpra3)
#dupl3.get_group(dupl_cpra3[37])
#set(hboc.loc[30]) ^ set(hboc.loc[634])

In [18]:
# index 293/294 (gr1) classes 3/2
# index 515/516 (gr6) classes 2/3
# index 3366/3367 (gr20) classes 2/3

# drop those entries:
for_drop2 = [176, 348, 466, 468, 492, 623, 4001, 1190, 1240, 1248, 1302, 1329, 1372, 1373, 1371, 1444, 1462, 
             1465, 1537, 3419, 3276, 801, 818, 851, 1045, 1061, 1120, 1125, 1135, 1651, 2778, 2905, 1936, 2044, 
             30, 3035]
hboc2.drop(index=for_drop2, inplace=True)

In [19]:
hboc2 = hboc2.rename(columns={'ART':'kind_hboc', 'KLASSIFIKATION':'classification_hboc', 
                              'ERFASSUNG':'aquisition_hboc', 'TASKFORCE_REVIEWED':'taskforce_review_hboc', 
                              'DATUM_TASKFORCE_REVIEW':'date_taskforce_hboc', 
                              'BEMERKUNG_TASKFORCE':'note_hboc', 'SONSTIGE_BEMERKUNGEN':'add_note_hboc', 
                              'SPLICE_PREDICTION_ALAMUT':'splice_pred_alamut_hboc', 
                              'PREDICTION_UMD_PREDICTOR':'prediction_hboc', 'KOMMENTAR':'comment_hboc', 
                              'LITERATUR_ERGEBNIS':'literature_hboc', 
                              'EVIDENZLEVEL_LITERATUR':'evidence_level_lit_hboc', 
                              'HANDLUNGSEMPFEHLUNG':'recomm_action_hboc', 'ID':'ID_hboc'})
hboc2 = hboc2[['#CHROM', 'POS', 'REF','ALT', 'ID_hboc', 'kind_hboc', 'classification_hboc', 'aquisition_hboc', 
               'taskforce_review_hboc', 'date_taskforce_hboc', 'note_hboc', 'add_note_hboc', 
               'splice_pred_alamut_hboc', 'prediction_hboc', 'literature_hboc', 'evidence_level_lit_hboc', 
               'comment_hboc', 'recomm_action_hboc']]

In [ ]:
vars_vep_floss_hboc = pd.merge(vars_vep_floss, hboc2, on=['#CHROM','POS','REF','ALT'], how='left')

### New SpliceAI scores (for variants without SplicaAI score)

In [ ]:
# previously as dataframe
splai = pd.read_pickle('DataFrame_new_SpliceAI')

In [ ]:
# change out missing SpliceIA score with newly generated SpliceAI score
new_splai = pd.merge(vars_vep_floss_hboc[vars_vep_floss_hboc['SpliceAI_pred_DP_AG'].isnull()].copy(), 
                     splai[['#CHROM', 'POS', 'REF', 'ALT', 'DS_AG', 'DS_AL', 'DS_DG', 'DS_DL', 'DP_AG', 'DP_AL', 
                            'DP_DG', 'DP_DL', 'SYMBOL']], on=['#CHROM', 'POS', 'REF', 'ALT'], how='left')
new_splai = new_splai.drop(columns=['SpliceAI_pred_DP_AG','SpliceAI_pred_DP_AL','SpliceAI_pred_DP_DG',
                                    'SpliceAI_pred_DP_DL','SpliceAI_pred_DS_AG','SpliceAI_pred_DS_AL',
                                    'SpliceAI_pred_DS_DG','SpliceAI_pred_DS_DL','SpliceAI_pred_SYMBOL'], )
new_splai = new_splai.rename(columns={i:'SpliceAI_pred_'+i for i in ['DS_AG','DS_AL','DS_DG','DS_DL','DP_AG',
                                                                     'DP_AL','DP_DG','DP_DL','SYMBOL']})
with_splai = vars_vep_floss_hboc[vars_vep_floss_hboc['SpliceAI_pred_DP_AG'].notnull()].copy()

In [ ]:
vars_vep_floss_hboc_splai = with_splai.append(new_splai)
vars_vep_floss_hboc_splai['#CHROM'] = pd.Categorical(vars_vep_floss_hboc_splai['#CHROM'],
                                      categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                  '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                      ordered=True)
vars_vep_floss_hboc_splai.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], inplace=True, ignore_index=True, 
                       ascending=True)

# save variants with all merged annotations together in one dataframe
vars_vep_floss_hboc_splai.to_pickle('all_annotated_variants')